In this document I'll create and modify variables necessary for my machine learning

In [1]:
import pandas as pd

In [2]:
hh = pd.read_csv("/Users/xavier/Desktop/DSPP/DS/Data-Science-1-Final-Project/Data_Wrangling/hh_2019.csv")

In [3]:
#in previous work with the data, I've noticed that hrhhid is almost unique but approximatley 5000 households share the same ID (duplexes or roommates I've been told)
#the two ID numbers have to be combines to have as an index
unique_id = hh['hrhhid'] + hh['hrhhid2']
hh.insert(0,"id",unique_id)

In [4]:
#drop irrelevant ID columns
hh = hh.drop(hh.columns[1:4], axis=1)

In [5]:
#replace all of the unknowns (usually as 99) with nul
hh = hh.replace(99,'')
hh = hh.replace(".",'')

In [6]:
#import other datasets
pop = pd.read_excel("/Users/xavier/Desktop/DSPP/DS/Data-Science-1-Final-Project/Other_Data/State_Population.xlsx")
ur_banks = pd.read_excel("/Users/xavier/Desktop/DSPP/DS/Data-Science-1-Final-Project/Other_Data/State_UR_and_Banks.xlsx")
st_conv = pd.read_excel("/Users/xavier/Desktop/DSPP/DS/Data-Science-1-Final-Project/Other_Data/State_ST_FIPS.xlsx")

In [7]:
#fix a weird spacing issue on pop
pop['State'] = pop['State'].str[1:]
pop.head()

,State,pop_2020
0,Alabama,5024279
1,Alaska,733391
2,American Samoa[10],49710
3,Arizona,7151502
4,Arkansas,3011524


In [8]:
#merge datasets on FIPS codes
fips_ur_banks = pd.merge(ur_banks,st_conv, on='ST')
fips_pop = pd.merge(pop,st_conv, on='State')
st_lvl = pd.merge(fips_pop,fips_ur_banks,on='FIPS',how='left')

In [9]:
#create a bank density and pull in the unemployment variable
st_lvl["ST_bnk_density"] = st_lvl["ST_NUM_BANKS"] / st_lvl["pop_2020"]
FIPS_dict = st_lvl[["FIPS","ST_bnk_density","ST_UR_19"]]
FIPS_dict.head()

,FIPS,ST_bnk_density,ST_UR_19
0,1,0.000022,3.0
1,2,0.000007,5.4
2,4,0.000002,4.9
3,5,0.000029,3.5
4,6,0.000004,4.2


In [10]:
hh['FIPS'] = hh['gestfips']

In [11]:
#add the new demographics to the dataset
hh = pd.merge(hh,FIPS_dict,on=['FIPS'],how='left')
hh.tail()

,id,gestfips,msa13,gtcbsast,hagele15,hbnkprev,hhincome,hhtenure,hhtype,hryear4,...,hincvolv2,hintaccv2,hbnkint,hunbnkrmv4,hhfamtyp,hsupresp,hsupwgtk,FIPS,ST_bnk_density,ST_UR_19
70198,521069004358376,56,0,3,0,,2,1,1,2019,...,,-1,-1,-1,1,0,0.0,56,0.000052,3.7
70199,710004565263948,56,0,3,0,,,1,6,2019,...,,-1,-1,-1,,0,0.0,56,0.000052,3.7
70200,614830557651116,56,0,3,0,,,2,6,2019,...,,-1,-1,-1,,0,0.0,56,0.000052,3.7
70201,561163008560585,56,0,3,0,,,2,6,2019,...,,-1,-1,-1,,0,0.0,56,0.000052,3.7
70202,654240932191116,56,0,3,0,,,2,6,2019,...,,-1,-1,-1,,0,0.0,56,0.000052,3.7


Here I create dummy variables for certain variables with 3-5 outcomes, to simplify regressions. These are categorical variables that really don't need to be categories.

In [12]:
#make an urban variable that is 1 if in or around a metro area and zero if not
hh['urban'] = hh['gtcbsast']
hh['urban'] = hh['urban'].replace(2,1)
hh['urban'] = hh['urban'].replace(3,0)
hh['urban'] = hh['urban'].replace(4,'')
hh = hh.drop(['gtcbsast'], axis=1)

In [13]:
hh['bank_prev'] = hh['hbnkprev']
hh['bank_prev'] = hh['bank_prev'].replace(2,0)
hh['bank_prev'] = hh['bank_prev'].replace(-1,0)

In [14]:
hh['unbanked'] = hh['hunbnk']
hh['unbanked'] = hh['unbanked'].replace(2,0)
hh = hh.drop(['hunbnk'], axis=1)

In [15]:
#Create Ed dummies
hh['no_diploma'] = hh['peducgrp']
hh['no_diploma'] = hh['no_diploma'].replace(2,0)
hh['no_diploma'] = hh['no_diploma'].replace(3,0)
hh['no_diploma'] = hh['no_diploma'].replace(4,0)
hh['no_college'] = hh['peducgrp']
hh['no_college'] = hh['no_college'].replace(2,1)
hh['no_college'] = hh['no_college'].replace(3,0)
hh['no_college'] = hh['no_college'].replace(4,0)
hh['degree'] = hh['peducgrp']
hh['degree'] = hh['degree'].replace(1,0)
hh['degree'] = hh['degree'].replace(3,1)
hh['degree'] = hh['degree'].replace(2,0)
hh['degree'] = hh['degree'].replace(4,0)

In [16]:
hh['poverty'] = hh['hhincome']
hh['poverty'] = hh['poverty'].replace(2,0)
hh['poverty'] = hh['poverty'].replace(3,0)
hh['poverty'] = hh['poverty'].replace(4,0)
hh['poverty'] = hh['poverty'].replace(5,0)

In [17]:
#create a binary that is 1 if repondent is a citizen and 0 if respondent is not
hh['citizen'] = hh['pnativ']
hh['citizen'] = hh['citizen'].replace(2,1)
hh['citizen'] = hh['citizen'].replace(3,0)

In [18]:
#binary where 1 is a native born and 0 is foreign born
hh['native_born'] = hh['pnativ']
hh['native_born'] = hh['native_born'].replace(2,0)
hh['native_born'] = hh['native_born'].replace(3,0)
hh = hh.drop(['pnativ'], axis=1)

In [19]:
#hagele15
#fix the homeownership variable to be a dummy
#1 has children 0 does not
hh['children'] = (hh['hagele15'] > 0).astype(int)

In [20]:
#hagele15
#fix the homeownership variable to be a dummy
#1 is under 25 0 is older
hh['under_25'] = (hh['prtage'] < 26).astype(int)

In [21]:
#hhtype
#fix family type to dummy
#1 is single mother 0 is not
hh['single_mother'] = hh['hhtype']
hh['single_mother'] = hh['single_mother'].replace(1,0)
hh['single_mother'] = hh['single_mother'].replace(3,0)
hh['single_mother'] = hh['single_mother'].replace(4,0)
hh['single_mother'] = hh['single_mother'].replace(5,0)
hh['single_mother'] = hh['single_mother'].replace(6,0)
hh['single_mother'] = hh['single_mother'].replace(2,1)

In [22]:
#fix the homeownership variable to be a dummy
#1 is homeowner 0 is not
hh['homeowner'] = hh['hhtenure']
hh['homeowner'] = hh['homeowner'].replace(2,0)
hh = hh.drop(['hhtenure'], axis=1)

In [23]:
#fix the unbanked variable to be a dummy
#1 is unemployed 0 is not
hh['unemployed'] = hh['pempstat']
hh['unemployed'] = hh['unemployed'].replace(1,0)
hh['unemployed'] = hh['unemployed'].replace(3,0)
hh['unemployed'] = hh['unemployed'].replace(2,1)

In [24]:
#fix volatility dummy
#1 is income volitility 0 is stablle
hh['inc_vol'] = hh['hincvolv2']
hh['inc_vol'] = hh['inc_vol'].replace(1,0)
hh['inc_vol'] = hh['inc_vol'].replace(2,1)
hh = hh.drop(['hincvolv2'], axis=1)

In [25]:
#fix the internet acces variable variable to be a dummy
#1 has internet access 0 does not
hh['internet'] = hh['hintaccv2']
hh['internet'] = hh['internet'].replace(2,0)
hh = hh.drop(['hintaccv2'], axis=1)

In [26]:
#fix the disability variable to be a dummy
#1 is udisabled 0 is not
hh['disability'] = hh['pdisabl_age25to64']
hh['disability'] = hh['disability'].replace(2,0)
hh = hh.drop(['pdisabl_age25to64'], axis=1)

In [27]:
#Convert continuous variables to legible names
hh['income'] = hh['hhincome']
hh['education'] = hh['peducgrp']
hh['age'] = hh['prtage']

In [28]:
#splid data by under 5 and over 5
#1 for White or Asian American or Pacific Islander, 0 for Other People of Color
hh['White_or_AAPI'] = hh['praceeth3']
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(1,0)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(2,0)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(3,1)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(4,0)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(5,1)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(6,1)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(7,0)

In [31]:
hh_small = hh[["unbanked","urban","homeowner","disability","degree","no_college","no_diploma","FIPS","ST_bnk_density","ST_UR_19","income","age","education","bank_prev","poverty","under_25","unemployed","citizen","native_born","White_or_AAPI","inc_vol","internet","children","single_mother"]]

In [ ]:
hh_small.head()

In [32]:
hh_small.to_csv("/Users/xavier/Desktop/DSPP/DS/Data-Science-1-Final-Project/Data_Wrangling/hh4.csv")

Here I create dummy variables based on clusered outcomes

In [ ]:
#identify clusters in age

In [ ]:
agegrp = pd.crosstab(hh.unbanked,hh.prtage)
agegrp = agegrp.transpose()
agegrp.reset_index(inplace=True)
agegrp.columns = ['age','num_banked','num_unbanked','num_NA']
agegrp['pct_unb'] = 100*agegrp['num_unbanked']/(agegrp['num_unbanked']+ agegrp['num_banked'])
agegrp

In [ ]:
from plotnine import *
import warnings
warnings.filterwarnings('ignore')
import plotnine as p9

In [ ]:
(p9.ggplot(data=agegrp, mapping=p9.aes(x='age', y='pct_unb'))+
 geom_point(size=3,alpha=.5,show_legend=True) +
 labs(x='Age', y='Percent Unbanked',color="",title="Percent Unbanked by Age")
)

Originally, I planned to bin age or at least create a few binary variables; now, however, it seems that age flows fairly linearly and doesn't need any binsing

In [ ]:
#visually identify any unbanked clusters in race
racegrp = pd.crosstab(hh.unbanked,hh.praceeth3)
racegrp = racegrp.transpose()
racegrp.reset_index(inplace=True)
racegrp.columns = ['race','num_banked','num_unbanked','num_NA']
racegrp['pct_unb'] = 100*racegrp['num_unbanked']/(racegrp['num_unbanked']+ racegrp['num_banked'])
racegrp['race_name'] = ['Black', 'Hispanic', 'Asian', 'Native', 'Pacific', 'White', 'Biracial']
racegrp

In [ ]:
(ggplot(racegrp)       
 + aes(x='race_name', y='pct_unb')    
 + geom_bar(stat="identity") # defining the type of plot to use
 + labs(title='Percent Unbanked by Race', x='Race', y='Percent Unbanked') 
)

For Race, I think we can justify Bins, especially since it is purely categorical

In [ ]:
#splid data by under 5 and over 5
#1 for White or Asian American or Pacific Islander, 0 for Other People of Color
hh['White_or_AAPI'] = hh['praceeth3']
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(1,0)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(2,0)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(3,1)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(4,0)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(5,1)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(6,1)
hh['White_or_AAPI'] = hh['White_or_AAPI'].replace(7,0)

# Export Main Dataset for Pre-Processing Review

In [ ]:
hh.to_csv("/Users/xavier/Desktop/DSPP/DS/Data-Science-1-Final-Project/Data_Wrangling/hh2.csv")

# Export Datasets for Slide Graphing

In [ ]:
#Export Already Initialized Data Groups
racegrp.to_csv("/Users/xavier/Desktop/DSPP/DS/Data-Science-1-Final-Project/Slides/graph_data/race_unbanked")
agegrp.to_csv("/Users/xavier/Desktop/DSPP/DS/Data-Science-1-Final-Project/Slides/graph_data/age_unbanked")

In [ ]:
#create dataset for education
edgrp = pd.crosstab(hh.unbanked,hh.peducgrp)
edgrp = edgrp.transpose()
edgrp.reset_index(inplace=True)
edgrp.columns = ['ed_level','num_banked','num_unbanked','num_NA']
edgrp['pct_unb'] = 100*edgrp['num_unbanked']/(edgrp['num_unbanked']+ edgrp['num_banked'])
edgrp['ed_name'] = ['No Diploma','Diploma','No Degree','Degree']
edgrp

In [ ]:
#create dataset for family type
ftgrp = pd.crosstab(hh.unbanked,hh.hhtype)
ftgrp = ftgrp.transpose()
ftgrp.reset_index(inplace=True)
ftgrp.columns = ['family_type','num_banked','num_unbanked','num_NA']
ftgrp['pct_unb'] = 100*ftgrp['num_unbanked']/(ftgrp['num_unbanked']+ ftgrp['num_banked'])
ftgrp['ft_name'] = ['Couple', 'F-Led Fam', 'M-Led Fam', 'F Ind', 'M Ind','Other']
ftgrp

In [ ]:
#create dataset for income
incgrp = pd.crosstab(hh.unbanked,hh.hhincome)
incgrp = incgrp.transpose()
incgrp.reset_index(inplace=True)
incgrp.columns = ['income','num_banked','num_unbanked','num_NA']
incgrp['pct_unb'] = 100*incgrp['num_unbanked']/(incgrp['num_unbanked']+ incgrp['num_banked'])
incgrp['inc_name'] = ['<15k', '15k-30k', '30k-50k', '50k-75k', '75k+','no response']
incgrp = incgrp.drop(5)
incgrp

In [ ]:
#export other variables
#Export Already Initialized Data Groups
edgrp.to_csv("/Users/xavier/Desktop/DSPP/DS/Data-Science-1-Final-Project/Slides/graph_data/ed_unbanked")
ftgrp.to_csv("/Users/xavier/Desktop/DSPP/DS/Data-Science-1-Final-Project/Slides/graph_data/ft_unbanked")
incgrp.to_csv("/Users/xavier/Desktop/DSPP/DS/Data-Science-1-Final-Project/Slides/graph_data/income_unbanked")